In [23]:
import pandas as pd
import numpy as np
#https://docs.google.com/spreadsheets/d/1OdjccfGlv3lsuiWgIAHbE8id91FpVaU2EsaZo5kknaA/edit?usp=sharing

In [10]:

from io import BytesIO
import requests
spreadsheet_id = '1OdjccfGlv3lsuiWgIAHbE8id91FpVaU2EsaZo5kknaA'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
df = pd.read_csv(BytesIO(r.content))


In [31]:
df.info()
df['date'] = pd.to_datetime(df['MMM-YY'], format='%Y-%m-%d')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 13 columns):
MMM-YY                  19104 non-null object
Emp_ID                  19104 non-null int64
Age                     19104 non-null int64
Gender                  19104 non-null object
City                    19104 non-null object
Education_Level         19104 non-null object
Salary                  19104 non-null int64
Dateofjoining           19104 non-null object
LastWorkingDate         1616 non-null object
Joining Designation     19104 non-null int64
Designation             19104 non-null int64
Total Business Value    19104 non-null int64
Quarterly Rating        19104 non-null int64
dtypes: int64(7), object(6)
memory usage: 1.9+ MB


In [32]:
def get_info(df):

    print("Head:")
    display(df.head())
    print()
    
    print("Info:")
    display(df.info())
    print()
    
    print("Description of quantitative parameters:")
    display(df.describe())
    print()
    
    print("Description of categorical parameters:")
    display(df.describe(include='object'))
    print()
    
    print("Columns with NaN values:")
    display(df.isna().sum())
    print()
    
    print("Shape:")
    display(df.shape)
    print()
    
    print("Number of duplicated rows:")
    display(df.duplicated().sum())  

In [33]:
get_info(df)

Head:


,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,date
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2,2016-01-01
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2,2016-02-01
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,2016-03-01
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1,2017-11-01
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1,2017-12-01



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 14 columns):
MMM-YY                  19104 non-null object
Emp_ID                  19104 non-null int64
Age                     19104 non-null int64
Gender                  19104 non-null object
City                    19104 non-null object
Education_Level         19104 non-null object
Salary                  19104 non-null int64
Dateofjoining           19104 non-null object
LastWorkingDate         1616 non-null object
Joining Designation     19104 non-null int64
Designation             19104 non-null int64
Total Business Value    19104 non-null int64
Quarterly Rating        19104 non-null int64
date                    19104 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(7), object(6)
memory usage: 2.0+ MB


None


Description of quantitative parameters:


...

MMM-YY                      0
Emp_ID                      0
Age                         0
Gender                      0
City                        0
Education_Level             0
Salary                      0
Dateofjoining               0
LastWorkingDate         17488
Joining Designation         0
Designation                 0
Total Business Value        0
Quarterly Rating            0
date                        0
dtype: int64


Shape:


(19104, 14)


Number of duplicated rows:


0

In [34]:
df['Emp_ID'].nunique()

2381

In [38]:
ids = df['Emp_ID'].unique()

In [35]:
dates = df['MMM-YY'].unique()
dates

array(['2016-01-01', '2016-02-01', '2016-03-01', '2017-11-01',
       '2017-12-01', '2016-12-01', '2017-01-01', '2017-02-01',
       '2017-03-01', '2017-04-01', '2017-08-01', '2017-09-01',
       '2017-10-01', '2016-07-01', '2016-08-01', '2016-09-01',
       '2016-10-01', '2016-11-01', '2016-04-01', '2016-05-01',
       '2016-06-01', '2017-05-01', '2017-06-01', '2017-07-01'],
      dtype=object)

In [14]:
df['Education_Level'].unique()

array(['Master', 'College', 'Bachelor'], dtype=object)

## Check whether education level changed

In [51]:
def education(df, id):
    temp = df.loc[df['Emp_ID'] == id]
    if temp['Education_Level'].nunique() > 1:
        return True
    else:
        return False

In [52]:
counter = 0
for i in ids:
    if (education(df, i)):
        counter = counter + 1
        
print(counter)

0


## Compare Designation and Salary

In [81]:
salary_ids = []
def salary(df, id):
    temp = df.loc[df['Emp_ID'] == id]
    if temp['Salary'].nunique() > 1:
        salary_ids.append(id)
        return True
    else:
        return False
    
designation_ids = []
def designation(df, id):
    temp = df.loc[df['Emp_ID'] == id]
    if temp['Designation'].nunique() > 1:
        designation_ids.append(id)
        return True
    else:
        return False

In [82]:
counter = 0
for i in ids:
    salary(df,i)
    designation(df,i)

In [85]:
print(salary_ids)
print(designation_ids)

[26, 54, 60, 98, 275, 307, 320, 368, 434, 537, 568, 580, 582, 638, 716, 789, 888, 1031, 1050, 1161, 1165, 1206, 1249, 1274, 1316, 1327, 1770, 1783, 1817, 1840, 1852, 1877, 1918, 2008, 2070, 2087, 2198, 2272, 2390, 2407, 2543, 2567, 2625, 2690]
[26, 54, 60, 98, 275, 307, 320, 368, 434, 537, 568, 580, 582, 638, 716, 789, 888, 1031, 1050, 1161, 1165, 1206, 1249, 1274, 1316, 1327, 1770, 1783, 1817, 1840, 1852, 1877, 1918, 2008, 2070, 2087, 2198, 2272, 2390, 2407, 2543, 2567, 2625, 2690]
